# Maîtriser l’utilisation des serveurs de calcul
## Les espaces de stockage
Il y a plusieurs types de stockage à l'Alliance de Recherche numérique du Canada :
* Personnel (`/home`)
* Temporaire local (`$SLURM_TMPDIR`)
* Temporaire réseau (`/scratch`)
* Projet partagé (`/project`)
* *Nearline* de longue durée (`/nearline`)
* [Dépôt fédéré de données de recherche](https://www.frdr-dfdr.ca/repo/) (DFDR)

Or, vos données ont plusieurs aspects :
* **Taille** : petits, gros, très gros fichiers
* **Nombre** : peu nombreux ou en très grand nombre
* **Transférabilité** : données regroupées et/ou compressées
* **Vie** : pendant une tâche, entre des tâches, d'un projet à l'autre, à archiver
* **Niveau d'accès** : données confidentielles, partagées, publiées

Le but de ce chapitre est de pouvoir considérer tous les aspects nécessaires à la gestion des données actives sur les différents espaces de stockage disponibles.

## Description de vos données
Au chapitre précédent, nous avons vu comment obtenir le nombre de fichiers de nos données locales, de même que la taille totale.

Sur une grappe de calcul, un **système de fichiers parallel**
[*Lustre*](https://docs.computecanada.ca/wiki/Tuning_Lustre/fr) ou *GPFS*
est surtout optimisé pour des fichiers de plus de 10 Mo,
c'est-à-dire pour des accès à large bande-passante. Ainsi, à l'échelle des
[grappes de calcul](https://docs.computecanada.ca/wiki/B%C3%A9luga#Stockage) :

* 1 Ko - 100 Ko : très petit fichier
* 1 Mo - 10 Mo : petit fichier
* 100 Mo - 1 Go : taille raisonnable
* 10 Go - 100 Go : gros fichier
* 1 To - 10 To : très gros fichier

À quantité totale égale d'octets, il est plus difficile pour *Lustre* de gérer un grand nombre de petits fichiers qu'un plus petit nombre de grands fichiers. Ainsi, du point de vue du système de stockage :

* Par dossier :
  * 1 à 100 fichiers par dossier : c'est raisonnable
  * 1000 à 5k fichiers par dossier : des lenteurs se font sentir
  * 10k fichiers et plus : il faut classer les fichiers dans des sous-dossiers
* Par espace [limité par un quota](https://docs.computecanada.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques) :
  * 1 à 500k fichiers : sauvegarde facilement à jour
  * 1M à 5M fichiers : sauvegarde potentiellement plus longue que 24h
  * 10M fichiers et plus : sauvegarde plus longue que 24h

### Transférabilité
#### Regroupement dans un fichier archive
Étant donné que le transfert ou la synchronisation de [plusieurs petits fichiers](https://docs.computecanada.ca/wiki/Handling_large_collections_of_files/fr) implique un accès fréquent aux métadonnées du système de stockage *Lustre*, il est donc préférable de [regrouper les fichiers](https://docs.computecanada.ca/wiki/Archiving_and_compressing_files/fr) d'un jeu de données avant tout transfert.
Pour ce faire, il existe plusieurs formats décodables sur les différentes grappes :

* WinZip ou Zip (`.zip`) - compressé
```
man zip  # q pour quitter
```
* 7-Zip (`.7z`) - compressé
```
man 7z  # q pour quitter
```
* Tape ARchive ou [TAR](https://docs.computecanada.ca/wiki/A_tutorial_on_%27tar%27/fr) (`.tar`) - non compressé, sauf si utilisé avec Gzip ou autre
```
man tar  # q pour quitter
```
* Disk ARchive ou [DAR](https://docs.computecanada.ca/wiki/Dar/fr) (`.dar`) - compression configurable
```
man dar  # q pour quitter
```

#### Compression
La [compression des fichiers](https://docs.computecanada.ca/wiki/Archiving_and_compressing_files/fr) a un **coût en temps CPU** à chaque fois que nous voulons lire les données ou les écrire.
* Ainsi, il vaut mieux utiliser la compression **lorsque c'est "avantageux"** en espace et/ou en temps.
  * Par exemple : décompression des fichiers sur le noeud de calcul plutôt que de lire chaque fichier de *Lustre*.

Concernant les fichiers binaires tels que FLAC, JPEG, PNG, etc., leur encodage **inclut déjà une compression** des données.
* Selon le type de données que vous utilisez dans votre domaine de recherche, il se pourrait qu'elles soient déjà compressées. À vous de vérifier.
* Habituellement, on gagne peu d'espace à compresser davantage ces fichiers.

#### Exercice - DAR
Dans un terminal :
```
dar -w -c dossiers -g images -g scripts
```
* `-w` : pour éviter un avertissement d'écrasement d'archive existante
* `-c` : pour créer l'archive `dossiers`
* `-g` : pour spécifier un fichier ou dossier à la fois

```
dar -l dossiers
```
* `-l` : pour lister les fichiers dans l'archive DAR `dossiers`
* Note : **ne pas** fournir le nom d'une *slice* avec l'extension `.1.dar`

```
dar -w -z -c dossiers -g images -g scripts
```
* `-z` : pour activer la compression
* Voir le résultat avec `dar -l dossiers`

```
dar -w -z -Z '*.png' -c dossiers -g images -g scripts
```
* `-Z '*.png'` : pour exclure les fichiers PNG
* Voir le résultat avec `dar -l dossiers`

Sur les grappes de calcul, il y a deux (2) types de compression :
1. Compression explicite :
  * Lorsque vous utilisez un outil de compression (`7z`, `bzip2`, `gzip`, `zip`, etc.)
  * Lorsque l'encodage permet de compresser les données (`dar`, bibliothèque `zlib`, etc.)
  * Lors d'un transfert avec `rsync`, il existe des options permettant de compresser automatiquement les données lors du transfert. Cela étant dit, le transfert risque d'être limité par la vitesse du processeur...
```
man rsync  # Chercher: /compress + n, Quitter: q
```
1. Compression implicite :
  * Lorsque le système de fichiers compresse automatiquement vos données avant de les écrire sur le système de stockage
    * La commande `du` montre l'espace réellement utilisée sur le disque
    * La commande `du --apparent-size` montre la taille normale du fichier
  * Lorsque le format du fichier de données inclut une compression des données. Par exemple :
    * [Hierarchical Data Format - HDF5](https://docs.computecanada.ca/wiki/HDF5/fr) - format standard pour données scientifiques
    * [Apache Parquet](https://parquet.apache.org/documentation/latest/) - stockage compressé de données tabulaires
      * Note: l'autre format, [Apache Feather](https://arrow.apache.org/docs/python/feather.html), ne compresse pas vraiment les données, sauf si on considère que c'est un format binaire prenant moins de place

#### Exercice - Espace utilisé
Dans un terminal Jupyter, essayez les commandes :
* `du -s ~`
* `du -s --apparent-size ~`
* Notez les différences, s'il y en a
* Notez à quel point cette commande peut surcharger le système de stockage; le survol récursif des fichiers constitue un grand nombre d'accès aux métadonnées de *Lustre*

En pratique, les grappes de calcul offrent des outils efficaces pour mesurer l'espace utilisé en fonction de l'espace alloué.
Nous verrons cela à la fin du chapitre.

#### Fichiers épars (sparse files)
Restez à l'affût des fichiers épars dont la faible taille sur le disque n'est pas représentative de la taille normale du fichier :
* Une certaine taille `T` est allouée pour le fichier. Par exemple : 1 To
* Seulement `t` octets sont réellement écrits dans le fichier (où `t << T`). Par exemple : 1 Go
* Ce type de fichiers est typiquement utilisé pour des volumes de stockage de machines virtuelles, ce qui économise de l'espace sur le serveur hôte
* Problème : lors d'un transfert ou d'une sauvegarde (backup), ce sont les `T` octets qui sont lus, ce qui peut être catastrophique si `T` est vraiment très grand
* **Solution** : les fichiers `.dar` permettent d'optimiser l'encapsulation de fichiers épars avant un transfert ou une copie des données

### Plan de gestion des données actives
Les données sur les grappes de calcul sont généralement présentes en attente d'être traitées, manipulées ou publiées, d'où la notion de *données actives*. Or, il y a une planification à faire et des questions à se poser.

#### Vie des données
* **Durée de vie variable** - avons-nous affaire à des données temporaires ou à des données persistantes?
  * Utilisées (à répétition ou non) pendant une seule tâche de calcul?
  * Utilisées entre deux tâches de calcul?
  * Utilisées pour plusieurs calculs par plusieurs personnes?
  * Peut-on importer à nouveau les données en cas de perte?
  * Peut-on recalculer les données en cas de perte? Si oui, à quel coût?
* **Fréquence d'accès** - combien de temps sépare les différents accès?
  * Utilisation courante (à chaque jour, semaine ou mois)?
  * Données à garder pour plus tard, par exemple pour reproduire les résultats?
  * Données à publier sur des dépôts de données de recherche?

Si aucune planification n'est faite, les données temporaires s'accumulent et prennent de la place dans l'espace alloué qui est de taille limitée. Si les données sont mal décrites, leur identification, leur compréhension et leur publication deviennent des tâches laborieuses pour les personnes qui en héritent.
```
cat .gitignore  # Fichiers à ignorer, car temporaires
cat README.md   # Description du projet
```

#### Niveaux d'accès
* Quel est le **niveau de confidentialité** de vos données?
  * **Qui** devrait avoir accès à quelles données?
  * **Où** pouvez-vous héberger vos données?
* **Quand** : qui devrait avoir accès à vos données à votre départ?

Référez-vous aux **politiques de votre université** concernant la gestion des données de recherche.
Dans tous les cas, sans précaution, les données que vous laissez sur les grappes de calcul risquent de ne pas être accessibles aux membres d'un laboratoire qui poursuivent leur recherche.

Dès que possible, mettez en place les accès nécessaires pour le [partage de vos données](https://docs.computecanada.ca/wiki/Sharing_data/fr).

## Les différents types de stockage
Stockage accessible selon le type de noeud :
* Noeud de connexion (ou interactif)
* Noeud de calcul (CPU ou GPU)

|     Stockage    | Connexion |  Calcul |
|-----------------|:---------:|:-------:|
|         `/home` |    Oui    |   Oui*  |
| `$SLURM_TMPDIR` |  **Non**  |   Oui   |
|      `/scratch` |    Oui    |   Oui   |
|      `/project` |    Oui    |   Oui   |
|     `/nearline` |    Oui    | **Non** |
`*` En lecture-seule sur Cedar

À propos des [différents espaces de stockage](https://docs.computecanada.ca/wiki/Storage_and_file_management/fr#Types_de_stockage).

### Votre dossier personnel (`/home`)
```
ls -a
ls -la
ls -ld $HOME
```

* **Point d'entrée** par défaut lors d'une connexion à une grappe de calcul
* [Quota d'espace relativement petit](https://docs.computecanada.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques), mais accepte un nombre de fichiers relativement grand
  * Idéal pour la [compilation et l'installation de logiciels](https://docs.computecanada.ca/wiki/Installing_software_in_your_home_directory/fr)

### Dossier temporaire local (`$SLURM_TMPDIR`)
```
ls -ld $SLURM_TMPDIR
salloc
  ls -ld $SLURM_TMPDIR
  df -h $SLURM_TMPDIR
  exit
```

* Stockage **local très rapide**, mais limité à la durée de la tâche de calcul
  * Faible latence à comparé *Lustre*
  * Grande bande-passante, en particulier pour les petits fichiers
  * **Données supprimées à la fin** de la tâche de calcul
  * Si plusieurs noeuds travaillent sur un même calcul parallèle, **chaque noeud a son propre dossier** `$SLURM_TMPDIR`
* Cas d'utilisation :
  * **Importation** de plusieurs **petits fichiers** qui seront utilisés à répétition lors d'un calcul
  * **Sauvegarde** de fichiers qui sont **constamment modifiés** - à exporter vers Lustre à la fin du calcul

### Espace réseau pour données temporaires (`/scratch`)
```
df -h /scratch
ls -ld $SCRATCH

# Enlever l'accès en lecture
chmod g-r $SCRATCH
ls -ld $SCRATCH

# Modifier le GID du répertoire
id
chgrp def-sponsor00 $SCRATCH
ls -ld $SCRATCH
```

```
# Création d'un dossier de résultats partagés
mkdir -p $SCRATCH/partage/resultats
ls -lR $SCRATCH

# Ajouter les permissions r (et x, s'il y a lieu) récursivement
chmod -R o+rX $SCRATCH/partage
ls -lR $SCRATCH
```

```
# Visiter le dossier partagé de quelqu'un d'autre
cd /scratch/userXY/partage
pwd
ls -l
cd $HOME
```

* Espace de stockage réseau de [grande capacité](https://docs.computecanada.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques) pour des **données temporaires**
  * **Pas** sauvegardé
  * [Purge mensuelle](https://docs.computecanada.ca/wiki/Scratch_purging_policy/fr) pour les données âgées de plus de 60 jours
* Performance variable selon l'utilisation de l'ensemble des utilisateurs
* Cas d'utilisation :
  * Utilisation de données **pendant quelques jours** seulement
  * Stocker temporairement des **résultats en grand nombre** de fichiers
  * Stocket des résultats **intermédiaires** qui sont **trop gros** pour `/project`

### Espace projet partagé (`/project`)
```
ls -ld /project
ls -ld /project/def-sponsor00
ls -l /project/def-sponsor00
```

* Espace de stockage réseau de [petite à grande capacité](https://docs.computecanada.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques) pour des **données de projet**
  * Un espace projet par défaut par groupe de recherche (sauf sur Niagara)
    * Petite augmentation d'espace projet **sur simple demande (jusqu'à 10 To)**
    * Quota plus élevé si [allocation spéciale](https://www.computecanada.ca/page-daccueil-du-portail-de-recherche/acces-aux-ressources/concours-dallocation-des-ressources/ressources-pour-les-groupes-de-recherche-crgr/?lang=fr)
  * Sauvegardé **quotidiennement**
    * Nombre de fichiers relativement limité
* Données de projet :
  * Potentiellement **partagées** - [configuration des ACLs](https://docs.computecanada.ca/wiki/Sharing_data/fr)
  * Durée de vie en fonction de la durée d'un projet
  * Typiquement plus importantes que les données temporaires
* Cas d'utilisation :
  * Stockage de jeux de données **réutilisés sur plusieurs mois et/ou par plusieurs personnes**
  * Stockage de **résultats finaux** coûteux à reproduire

### Stockage très longue durée (`/nearline`)
* Malgré une interface de stockage sur disque (on peut voir les fichiers avec la commande `ls`), les données les plus anciennes dans [`/nearline` sont probablement sur ruban](https://docs.computecanada.ca/wiki/Using_nearline_storage/fr)
  * Voir les commandes pour [diagnostiquer l'état des fichiers](https://docs.computecanada.ca/wiki/Using_nearline_storage/fr#Transf.C3.A9rer_des_donn.C3.A9es_.C3.A0_partir_de_.2Fnearline) dans `/nearline`
  * D'une part, la migration de données sur ruban réduit l'espace utilisé sur les disques - économie d'argent pour le stockage
  * D'autre part, chaque lecture de fichier migré sur ruban créera **une requête bloquante** causant un temps de réponse de quelques minutes à quelques heures (si le système d'archivage est surchargé de requêtes)
    * Voilà pourquoi il est impératif d'y sauvegarder un faible nombre de très grands fichiers
    * **À éviter** : y copier de nombreux petits fichiers avant de les regrouper dans un fichier d'archive
* Cas d'utilisation :
  * Les commandes `7z`, `dar`, `tar` et `zip` devraient uniquement regrouper des fichiers provenant de `/project` ou de `/scratch`
  * Stockage de données importantes **qui ne seront pas utilisées pendant plusieurs mois**

### Exemples de cycles de vie des données
#### Exemple de pipeline 1
![Pipeline 1](images/data-flow-1.png)

Description des différentes étapes :
* Téléchargement des données dans `/scratch`
  * Pour une utilisation de quelques jours à quelques semaines
  * Nul besoin de les protéger davantage
* Soumettre une série de tâches de calcul
  * Une tâche par fichier dans `dossier` dans la partition `/scratch`
* Le script de tâche est dans la partition `/home`
  * Utilisation des variables `$FIC` et `$SLURM_TMPDIR` pour copier le fichier à traiter localement sur le noeud de calcul
  * Se déplacer dans le dossier local
  * Y configurer un environnement Python
  * Exécuter le code Python présent dans `/home`, fournir le nom du fichier à traiter et rediriger toutes les sorties dans un fichier local
  * Rapatrier le fichier de résultats dans un dossier dans `/scratch`
* Post-traitement - traiter tous les fichiers de résultats afin de ne garder que l'essentiel dans `/project`

#### Exemple de pipeline 2
![Pipeline 2](images/data-flow-2.png)

Description des différentes étapes :
* Téléchargement des données dans `/project`
  * Pour un partage des données
  * Pour une réutilisation à moyen-long terme
* Soumettre une série de tâches de calcul
  * Une tâche par *cas* (un sous-dossier de données) dans `dossier` dans la partition `/project`
* Le script de tâche est dans la partition `/home`
  * Utilisation des variables `$CAS` et `$SLURM_SUBMIT_DIR` pour le chemin complet vers le *cas* à étudier (situé dans `/project`)
  * Se déplacer dans le dossier local `$SLURM_TMPDIR` du noeud de calcul
  * Exécution de scripts présents dans `/home`
  * Toute la trace d'exécution est sauvegardée dans un fichier créé dans `$SLURM_TMPDIR`
  * À chaque itération globale, tous les fichiers archives dans le sous-dossier du *cas* (présent dans `/project`) sont traités :
    * Le traitement consiste à extraire localement les données groupées et compressées
    * Les données extraites sont dans un dossier local `donnees`
  * Rapatrier le fichier de résultats dans un dossier dans `/scratch`
* Post-traitement - traiter tous les fichiers de résultats :
  * Conversion en format CSV dans `/scratch`
  * Regrouper les fichiers CSV dans la partition `/project` où il y a de la compression implicite

#### Exercice - Exécution d'un pipeline
* Étudier les scripts :
  * `scripts/nbody-pipeline.sh`
  * `scripts/nbody-traitement.sh`
* Lancer le pipeline avec la commande `bash scripts/nbody-pipeline.sh`
* Surveiller les tâches avec `squeue -u $USER`
* Trouver les fichiers créés dans `/scratch/$USER/resultats`
* Lancer le post-traitement avec la commande `bash scripts/nbody-traitement.sh`
* Trouver le fichier TSV dans l'espace projet

## Gestion du stockage
Avec le temps, les données s'accumulent. Il devient alors nécessaire de surveiller l'espace utilisé, de même que le nombre de fichiers qui s'y trouvent.
```
df -h /project
df -hi /project
```

* [La commande `diskusage_report`](https://docs.computecanada.ca/wiki/Storage_and_file_management/fr#Introduction) permet de générer un court rapport sur l'espace utilisé et le nombre de fichiers de chacun des espaces de stockage auxquels vous avez accès
* À chaque jour, un rapport de consommation de l'espace `/project` par utilisateur est produit :
  * Sur Béluga : dans `/project/.stats/<nom-allocation>`
  * Sur Cedar : dans `/project/.stats/<nom-allocation>.json`
  * Disponible sur demande pour Graham et Niagara
* Sur Béluga, il existe aussi un outil plus avancé : [`diskusage_explorer`](https://docs.computecanada.ca/wiki/Diskusage_Explorer/fr)
  * Utilise les données en format SQLite générées quotidiennement dans `/project/.duc_databases/`
  * Permet d'avoir des statistiques de consommation du stockage par répertoire, ce qui évite d'utiliser la commande `du -sh *`

En cas de données inaccessibles :
* Une professeure ou un professeur peut demander à ce que les données soient supprimées
* Par contre, pour obtenir l'accès aux données, il faut avoir le consentement de la personne qui les a isolées (probablement involontairement)
  * En cas d'absence de réponse, c'est la politique de l'université du groupe de recherche qui permettra ou non d'obtenir l'accès aux données
  
Dans tous les cas, il vaut mieux prévenir, et ce, dès l'importation de données sur les grappes de calcul.